In [0]:
df = spark.read.json("/databricks-datasets/iot-stream/data-device/")
df.display()


In [0]:
%sql
CREATE TABLE IF NOT EXISTS sensor_events (
  deviceId STRING,
  timestamp STRING,
  location STRUCT<
      country: STRING,
      coords: STRUCT<lat: DOUBLE, lon: DOUBLE>
  >,
  readings ARRAY<STRUCT<
      type: STRING,
      value: DOUBLE,
      unit: STRING
  >>,
  metadata STRUCT<
      firmware: STRING,
      battery: INT,
      tags: ARRAY<STRING>
  >,
  ingest_id STRING,
  source_system STRING
)
USING DELTA;


In [0]:
%sql
INSERT INTO sensor_events VALUES
(
  "sensor-001",
  "2025-01-01T10:00:00Z",
  named_struct(
    "country", "UK",
    "coords", named_struct("lat", 52.2053, "lon", 0.1218)
  ),
  array(
    named_struct("type", "temperature", "value", 21.5, "unit", "C"),
    named_struct("type", "humidity", "value", 48.2, "unit", "%")
  ),
  named_struct(
    "firmware", "v1.4.2",
    "battery", 87,
    "tags", array("production", "east-region")
  ),
  "ing-001",
  "iot_gateway"
),
(
  "sensor-002",
  "2025-01-01T10:00:05Z",
  named_struct(
    "country", "UK",
    "coords", named_struct("lat", 52.2405, "lon", 0.1557)
  ),
  array(
    named_struct("type", "temperature", "value", 19.8, "unit", "C"),
    named_struct("type", "pressure", "value", 101.2, "unit", "kPa")
  ),
  named_struct(
    "firmware", "v1.4.2",
    "battery", 92,
    "tags", array("test-unit")
  ),
  "ing-002",
  "iot_gateway"
);


In [0]:
%sql
SELECT explode(readings) from sensor_events

More manual way

In [0]:
%sql
create table sensor_events_manual
( deviceid int,
storelocation string,
workingjsoncolumn string)

In [0]:
%sql
INSERT INTO sensor_events_manual values(
1,
'Bury st Edmunds',
'{ "deviceId": "sensor-001", "timestamp": "2025-01-01T10:00:00Z", "location": { "country": "UK", "coords": {"lat": 52.2053, "lon": 0.1218} }, "readings": [ {"type": "temperature", "value": 21.5, "unit": "C"}, {"type": "humidity", "value": 48.2, "unit": "%"} ], "metadata": { "firmware": "v1.4.2", "battery": 87, "tags": ["production", "east-region"] } }'
),
(2,
'Bury st Edmunds',
'{ "deviceId": "sensor-002", "timestamp": "2025-01-01T10:00:05Z", "location": { "country": "UK", "coords": {"lat": 52.2405, "lon": 0.1557} }, "readings": [ {"type": "temperature", "value": 19.8, "unit": "C"}, {"type": "pressure", "value": 101.2, "unit": "kPa"} ], "metadata": { "firmware": "v1.4.2", "battery": 92, "tags": ["test-unit"] } }'
)

## Get the STRUCT text output by using schema_of_json

In [0]:
%sql
select schema_of_json(workingjsoncolumn) from sensor_events_manual

Notice the : use in the query below. It utilzes spark built in functionality to interact directly with data stored as json

In [0]:
%sql
select workingjsoncolumn:deviceId
from sensor_events_manual

## Using the STRUCT structure in the python query
Way more performant  
Schema is enforced  
Reduced Flexibility  

In [0]:
from pyspark.sql.functions import from_json, col

jsonstruct = 'STRUCT<deviceId: STRING, location: STRUCT<coords: STRUCT<lat: DOUBLE, lon: DOUBLE>, country: STRING>, metadata: STRUCT<battery: BIGINT, firmware: STRING, tags: ARRAY<STRING>>, readings: ARRAY<STRUCT<type: STRING, unit: STRING, value: DOUBLE>>, timestamp: STRING>'
dfsense = spark.table("sensor_events_manual")
df_parsed2 = dfsense.withColumn(
    "parsed",
    from_json(col("workingjsoncolumn"), jsonstruct)
)
df_parsed2.write.saveAsTable("sensor_events_parsed")

## Using the STRUCT structure in the SQL query

Way more performant  
Schema is enforced  
Reduced Flexibility  


In [0]:
%sql
SELECT * ,
  from_json(
    workingjsoncolumn,
    'STRUCT<deviceId: STRING, location: STRUCT<coords: STRUCT<lat: DOUBLE, lon: DOUBLE>, country: STRING>, metadata: STRUCT<battery: BIGINT, firmware: STRING, tags: ARRAY<STRING>>, readings: ARRAY<STRUCT<type: STRING, unit: STRING, value: DOUBLE>>, timestamp: STRING>'
  ) AS parsed
FROM sensor_events_manual

# Use the new parsed table to query the parsed column

In [0]:
%sql
SELECT parsed.deviceid,
parsed.location.country,
parsed.metadata.battery,
parsed.metadata.firmware,
parsed.metadata.tags,
parsed.readings.type,
parsed.readings.value,
parsed.readings.unit,
parsed.timestamp
 from sensor_events_parsed

## Now use the explode
Having its own row for each element of the array

In [0]:
%sql
CREATE OR REPLACE TABLE sensor_events_exploded
AS 
SELECT deviceid, storelocation
, explode_outer(parsed.metadata.tags) AS tag
, explode_outer(parsed.readings.value) AS readingvalue
, parsed.metadata.tags as tagsarray
, parsed as fulljson
FROM sensor_events_parsed;
    
SELECT * FROM sensor_events_exploded;